In [ ]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        #random.seed(random_seed)
        #random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


In [ ]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

In [ ]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(0.45)
        )

    def forward(self, x):
        x = self.block(x)
        return x

##ref:https://www.cnblogs.com/picassooo/p/12556293.html
class LSTMnet(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(LSTMnet, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, 3, batch_first=True, dropout=0.45, bidirectional=True)
        self.fc = nn.Sequential(
            BasicBlock(2*hidden_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, n_class)
        )
 
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x


In [ ]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 1   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.95   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 910226          # random seed
batch_size = 1024        # batch size
num_epoch = 35         # the number of training epoch
change_lr_epoch = 30
learning_rate = 1e-3      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 1          # the number of hidden layers
hidden_dim = 280           # the hidden dim


In [ ]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data(split='val', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3257


3257it [00:23, 141.01it/s]


[INFO] train set
torch.Size([2010837, 39])
torch.Size([2010837])
[Dataset] - # phone classes: 41, number of utterances for val: 172


172it [00:01, 155.84it/s]


[INFO] val set
torch.Size([105957, 39])
torch.Size([105957])


In [ ]:
model = LSTMnet(in_dim=input_dim, hidden_dim=hidden_dim, n_layer=hidden_layers, n_class=41).to(device)
print(model)

LSTMnet(
  (lstm): LSTM(39, 280, num_layers=3, batch_first=True, dropout=0.45, bidirectional=True)
  (fc): Sequential(
    (0): BasicBlock(
      (block): Sequential(
        (0): Linear(in_features=560, out_features=280, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.45, inplace=False)
      )
    )
    (1): BasicBlock(
      (block): Sequential(
        (0): Linear(in_features=280, out_features=280, bias=True)
        (1): ReLU()
        (2): BatchNorm1d(280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Dropout(p=0.45, inplace=False)
      )
    )
    (2): Linear(in_features=280, out_features=41, bias=True)
  )
)


In [ ]:
# create model, define a loss function, and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.02)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=8e-4)
#lambda1 = lambda epoch: 0.1**int((epoch-1)/change_lr_epoch) ###if epoch > lr_epoch else 1
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda1) 
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=8, T_mult=2, eta_min=learning_rate/10) 
step_size = 20
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    print("lr = "+str(scheduler.get_last_lr()[0])) 
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        #features = features.unsqueeze(0)
        #print(features.size())
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    scheduler.step()
    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device)
            #features = features.unsqueeze(0)
            labels = labels.to(device)
            outputs = model(features)
            
            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    if val_acc/len(val_set) > best_acc:
        best_acc = val_acc/len(val_set)
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc:.5f}')


lr = 0.001


100%|██████████| 104/104 [00:10<00:00, 10.34it/s]


[001/035] Train Acc: 0.60605 Loss: 1.34721 | Val Acc: 0.71872 loss: 0.92873
saving model with acc 0.71872
lr = 0.001


100%|██████████| 104/104 [00:10<00:00, 10.28it/s]


[002/035] Train Acc: 0.72396 Loss: 0.93444 | Val Acc: 0.76283 loss: 0.78502
saving model with acc 0.76283
lr = 0.001


...

[033/035] Train Acc: 0.94444 Loss: 0.15825 | Val Acc: 0.83332 loss: 0.77104
saving model with acc 0.83332
lr = 0.0001


100%|██████████| 104/104 [00:10<00:00, 10.33it/s]


[034/035] Train Acc: 0.94581 Loss: 0.15371 | Val Acc: 0.83178 loss: 0.78224
lr = 0.0001


100%|██████████| 104/104 [00:10<00:00, 10.20it/s]

[035/035] Train Acc: 0.94705 Loss: 0.14908 | Val Acc: 0.83172 loss: 0.79697


In [ ]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

23

In [ ]:
# load data
test_X = preprocess_data(split='test', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:05, 150.05it/s]

[INFO] test set
torch.Size([527364, 39])


In [ ]:
# load model
model = LSTMnet(in_dim=input_dim, hidden_dim=hidden_dim, n_layer=hidden_layers, n_class=41).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

100%|██████████| 516/516 [00:49<00:00, 10.37it/s]


In [ ]:
with open('prediction_new.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))